In [1]:
import os
os.chdir('/home/megatron/work')

import numpy as np
import pandas as pd
import megatron
from sklearn.preprocessing import StandardScaler

In [2]:
# from dataframe, lazy

df = pd.DataFrame({'a': np.arange(50), 'b': np.arange(50), 'c': np.arange(50)})
G = megatron.Graph()

col_nodes = megatron.adapters.input.from_dataframe(df, G)

T = megatron.transforms.Add()
out = T([col_nodes['a'], col_nodes['b'], col_nodes['c']])

result = G.run(out, df, cache_result=False)
print(result.shape)

(50,)


In [4]:
# from dataframe, eager

df = pd.DataFrame({'a': np.arange(50), 'b': np.arange(50), 'c': np.arange(50)})
G = megatron.Graph()

col_nodes = megatron.adapters.input.from_dataframe(df, G, eager=True)

T = megatron.transforms.Add()
result = T([col_nodes['a'], col_nodes['b'], col_nodes['c']])

print(result.output.shape)

(50,)


In [2]:
# lazy

G = megatron.Graph(cache_dir='megatron/cache')
X = megatron.Input(G, 'X', (10,))
Y = megatron.Input(G, 'Y', (10,))

T = megatron.transforms.SklearnTransformation(StandardScaler())
X1 = T(X)
Y1 = T(Y)
X2 = megatron.transforms.TimeSeries(window_size=5)(X1)
Y2 = megatron.transforms.TimeSeries(window_size=5)(Y1)
Z = megatron.transforms.Add()([X2, Y2])

data = {'X': np.random.random((50, 10)),
        'Y': np.random.random((50, 10))}

result = G.run(Z, data, cache_result=True)
print(result.shape)

# test caching
result = G.run(Z, data, cache_result=True)
print(result.shape)

G.save('/home/megatron/work/test.p')

(45, 5, 10)
Loading node number 6 in path from cache
(45, 5, 10)


In [3]:
# eager

data = {'X': np.random.random((50, 10)),
        'Y': np.random.random((50, 10))}

G = megatron.Graph()
X = megatron.Input(G, 'X', (10,))(data['X'])
Y = megatron.Input(G, 'Y', (10,))(data['Y'])
X1 = megatron.transforms.TimeSeries(window_size=5)(X)
Y1 = megatron.transforms.TimeSeries(window_size=5)(Y)
Z = megatron.transforms.Add()([X1, Y1])

print(X.output.shape)
print(Z.output.shape)

(50, 10)
(45, 5, 10)


In [ ]:
# reloading a graph

G2 = megatron.load('/home/megatron/work/test.p')
G2.run('Add:0', data, cache_result=True).shape